# Project: Data Warehouse
Samuel Botter Martins

## Set up AWS resources

This jupyter notebook aims to setup _all AWS resources_ for the project. This includes creating:
- an IAM role and assigned to my IAM user
- creating a Redshift cluster with a default database
- testing the connection accessing the database

I followed almost the same steps as in Exercise 2 on _Infrastructure as Code_ but with some adaptations.

In [ ]:
import pandas as pd
import configparser
import boto3
import json

## Before we start
First, I followed the same steps as in the Exercise of IaC and created a new IAM user in my AWS account with _AdministrationAccess_. <br/>
I then used my _security credentials (access keys)_ to follow this project.

## 1. Load DWH Parameters from a configuration file
My AWS configuration is placed in the `dwh.cfg` file. For security reasons, I have just provided a template of this file called `dwh_template.cfg`..

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# AWS
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')
REGION = config.get('AWS', 'REGION')

# IAM
IAM_ROLE_NAME = config.get('IAM', 'ROLE_NAME')

# DWS
DWH_CLUSTER_TYPE = config.get('DWH', 'DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH', 'DWH_NUM_NODES')
DWH_NODE_TYPE = config.get('DWH', 'DWH_NODE_TYPE')

DWH_CLUSTER_IDENTIFIER = config.get('DWH', 'DWH_CLUSTER_IDENTIFIER')
DWH_DB = config.get('DWH', 'DWH_DB')
DWH_DB_USER = config.get('DWH', 'DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH', 'DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH', 'DWH_PORT')

In [ ]:
# show the loaded parameters
# uncomment the code below to show your credentials

# pd.DataFrame({
#     "Param": ['KEY', 'SECRET', 'REGION', 'IAM_ROLE_NAME', 'DWH_CLUSTER_TYPE', 'DWH_NUM_NODES', 'DWH_NODE_TYPE', 'DWH_CLUSTER_IDENTIFIER', 'DWH_DB', 'DWH_DB_USER', 'DWH_DB_PASSWORD', 'DWH_PORT'],
#     "Value": [KEY, SECRET, REGION, IAM_ROLE_NAME, DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT]
# })


## 2. IAM

### 2.1 Creating a client for IAM
Creating a python client to access AWS IAM from the user specified in the `dwg.cfg` configuration file.

In [ ]:
iam = boto3.client('iam',
                   region_name=REGION,
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

### 2.2 IAM Role
#### 2.2.1 *Create an IAM Role*
This makes Redshift able to access the S3 bucket (_ReadOnly_).

In [ ]:
try:
    print("Creating a new IAM Role") 
    
    dwh_role = iam.create_role(
                    Path='/',
                    # name given in my configuration file
                    RoleName=IAM_ROLE_NAME,
                    Description = "Allows Redshift clusters to call AWS services on your behalf.",
                    AssumeRolePolicyDocument = json.dumps({
                        'Statement': [{
                            'Action': 'sts:AssumeRole',
                            'Effect': 'Allow',
                            'Principal': {'Service': 'redshift.amazonaws.com'}
                        }],
                        'Version': '2012-10-17'
                    })
                )    
except Exception as e:
    print(e)

To check this created _role_:
- Go to _IAM_ dashboard on AWS.
- In the navigation panel, choose **Rules**
- You will see the _created role_ in the list of Roles

Click on this _role_ to see its properties. <br/>
Note that **there is no policy assigned to this role yet**. That's exactly what we're going to do now.

To _**delete**_ this role, select it in the list and delete it!

#### 2.2.2 *Assigning _policies_ to an IAM Role*
We will assign a policy to our role that allows Redshift to:
    - `AmazonS3ReadOnlyAccess`: access S3 buckets (ReadOnly).
    - `AmazonRedshiftQueryEditor`: access Redshift Query Editor.
      - Required to use the _Query Editor_ on AWS and the _SQL Workbench_ tool.

In [ ]:
print("Attaching Policy: AmazonS3ReadOnlyAccess")
iam.attach_role_policy(
            RoleName=IAM_ROLE_NAME,
            PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
)['ResponseMetadata']['HTTPStatusCode']

The return `200` means that worked!

In [ ]:
print("Attaching Policy: AmazonS3ReadOnlyAccess")
iam.attach_role_policy(
            RoleName=IAM_ROLE_NAME,
            PolicyArn="arn:aws:iam::aws:policy/AmazonRedshiftQueryEditor"
)['ResponseMetadata']['HTTPStatusCode']

Go again in the properties of the role in AWS to verify that the policies created were correctly assigned to it!

#### 2.2.3 *Get the IAM role ARN (Amazon Resource Names)"*

In [ ]:
role_arn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(role_arn)

An **ARN** _uniquely identifies_ AWS resources. In this case, we have the ARN for our function!

We will use this **ARN** when creating the Redshift cluster.

To use this function in future code, we saved it as a setting in our `dwh.cfg` configuration file: `[IAM] ROLE_ARN`.

<br/> <br/>

## 3. Redshift Cluster
### 3.1 Creating a client for Redshift

In [ ]:
redshift = boto3.client('redshift',
                        region_name=REGION,
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

### 3.2 Creating a **Redshift Cluster**
See the [official docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift/client/create_cluster.html#).

We need to create the **first database** when the cluster is created using the `DBName` parameter. To create additional databases after the cluster is created, connect to the cluster with an SQL client and use SQL commands to create a database. See the [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift/client/create_cluster.html#).

In [ ]:
try:
    response = redshift.create_cluster( 
                    # Data Warehouse specs
                    ClusterType=DWH_CLUSTER_TYPE,
                    NodeType=DWH_NODE_TYPE,
                    NumberOfNodes=int(DWH_NUM_NODES),
        
                    # first database of the cluster
                    DBName=DWH_DB,
                    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
                    MasterUsername=DWH_DB_USER,
                    MasterUserPassword=DWH_DB_PASSWORD,

                    # my user's role to manage this Redshift (for S3)
                    IamRoles=[role_arn]
    )
except Exception as e:
    print(e)

To check the created Redshift cluster:
- Access the _Amazon Redshift_ dashboard on AWS.
- Select the considered _region_ from the menu bar (in my case _us-east-1_)
- In the navigation pane, choose **Clusters**
- You will see the _cluster created_ in the _Clusters_ list
- It may take some time to finish creating the cluster (_Available_ status).

We can check the **cluster _status_** on the cluster dashboard in AWS Redshift. When the status becomes **Available**, follow the instructions below!

<br/><br/>

Click on this _cluster_ to see its **properties**. <br/>
We can see the _cluster specifications_ and _other information_ on **General information**, such as:
- cluster identification
- Cluster ARN
- Node type
- Number of nodes
- Endpoint, etc

We can manually copy any of this information to use in our future code, but we'll do it _directly_ from the code.

In the **Properties tab** we have other useful information:
- Database configurations
- Network and security settings
- Associated IAM roles 
  - **Check our role there**
- Node IP addresses
  - This is interesting because we can see _all the nodes_ needed (in our case 4) and the _leader_.

#### Getting the cluster properties by code
Instead of opening the AWS panel to get the cluster properties, we can do this in code.

In [ ]:
# describe cluster properties with identifier DWH_CLUSTER_IDENTIFIER
# from our redshift python client
cluster_props = redshift.describe_clusters(
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [ ]:
cluster_props

In [ ]:
# print only a few properties as a pandas dataframe
def pretty_redshift_props(props):
    prop_keys_to_show = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    
    x = [(k, v) for k,v in props.items() if k in prop_keys_to_show]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

pretty_redshift_props(cluster_props)

<br/>

#### Deleting a cluster
To _**delete**_ this cluster, go to the AWS Redshift cluster dashboard, select the cluster in the list and delete it! Otherwise, do that in code (see the jupyter notebook `teardown_AWS_resources.ipynb`).

<h3> 3.3 Get the cluster <font color='red'> endpoint (host) and role ARN </font> </h3>
<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available".</font>

In [ ]:
DWH_ENDPOINT = cluster_props['Endpoint']['Address']
DWH_ROLE_ARN = cluster_props['IamRoles'][0]['IamRoleArn']

print(f"DWH_ENDPOINT: {DWH_ENDPOINT}")
print(f"DWH_ROLE_ARN: {DWH_ROLE_ARN}")

Note that this information can be retrieved from the cluster properties on the AWS platform, as we did earlier. The `DWH_ROLE_ARN` is exactly the same `ROLE_ARNA` we got before.

To make our ETL codes easier, I will save the `DWH_ENDPOINT` in the `dwh.cfg` file.

### 3.4 Open an incoming **TCP port** _to access_ the cluster endpoint
To access the cluster through its endpoint, we need to open an inbound TCP port.

In [ ]:
# Get a EC2 client
ec2 = boto3.resource('ec2',
                     region_name=REGION,
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)

In [ ]:
try:
    vpc = ec2.Vpc(id=cluster_props['VpcId'])

    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        # information insider the file `dwh.cfg`
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

### 3.5 Checking cluster connection
The course used `pyscopg2` to connect to databases in Redshift. However, **it is outdated**.

Instead, we will use the `redshift_connector` package according to the [official AWS tutorial](https://docs.aws.amazon.com/redshift/latest/mgmt/python-connect-examples.html).
   it is now out of date to connect to redshift.

In [ ]:
!pip install redshift_connector

In [ ]:
import redshift_connector

conn = redshift_connector.connect(
     host=DWH_ENDPOINT,
     database=DWH_DB,
     port=DWH_PORT,
     user=DWH_DB_USER,
     password=DWH_DB_PASSWORD
  )

conn

#### **Connecting by SQL Workbench**
Install and configure _SQL Workbench_ to access your _Redshift Cluster_. I provided a simple tutorial for that in the [SQL_workbench_on_Redshift.md](./SQL_workbench_on_Redshift.md) file.

## 4. READY FOR ETL
If everything worked so far, then now we can run our ETL process!

## 5. Cleaning the resources by code
Keeping all active costs of created resources! So after finishing the project, running ETC, etc, we need to clean up/delete _all_ resources allocated in AWS. <br/>
One option is to do this _manually_ on the AWS platform. However, we can do this by code!

I created the jupyter-notebook `tear_down_AWS_resources.ipynb` to clean up/dismount all AWS resources directly from code.